## Simple simulation 

This is for testing propouses

In [10]:
import os
import subprocess
from pymavlink import mavutil

from vehicle_logic import VehicleLogic

## Kill all related processes

In [19]:
for process in ["QGroundControl", "gazebo", "sim_vehicle.py", "arducopter", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

# Choose Simulator

In [12]:
simulator ='gazebo' #'QGroundControl' # 

## Set simulator

In [13]:
if simulator == 'QGroundControl':
    vis_path = os.path.expanduser("~/QGroundControl.AppImage")
    vis_cmd =[vis_path]
elif simulator == 'gazebo':
    vis_path = os.path.expanduser("~/ardupilot_gazebo/worlds/iris_arducopter_runway.world")
    vis_cmd = ["gazebo", "--verbose", vis_path] 

In [14]:
vis_process = subprocess.Popen(
    vis_cmd,
    stdout=subprocess.DEVNULL,  # Suppress standard output
    stderr=subprocess.DEVNULL,  # Suppress error output
    shell=False  # Ensure safety when passing arguments
)


# Set Vehicle Autopilot

In [15]:
# Define the command string
sim_vehicle_path = os.path.expanduser("~/ardupilot/Tools/autotest/sim_vehicle.py")
vehicle_cmd = f"python3 {sim_vehicle_path} -v ArduCopter" 
if simulator == 'gazebo':
    vehicle_cmd += ' -f gazebo-iris'
# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--', 'bash', '-c', 'pytho...>

## Create Plan

In [16]:
takeoff_pos=(-35.3633286,149.1652227,585,353)

# Plan
alt = 5
dist = 10
wps=[(0, 0, -alt), #takeoff point
    (0,dist, -alt),
    (dist, dist, -alt),
    (dist, -dist, -alt),
    (0, -dist, -alt),
    (0, 0, -alt)]
plan=['check_prearm','check_pos_est','mode_stabilize','mode_guided','arm','takeoff','fly','land']

uavs_arg = []
uav_arg={'takeoff_pos':takeoff_pos,
        'waypoints':wps,
        'plan': plan,
        'wp_margin': 0.5}

## Create UavLogic

In [17]:
i=0
connection = mavutil.mavlink_connection(f'udp:127.0.0.1:{14551+10*i}')
connection.wait_heartbeat()
uav=VehicleLogic(connection,**uav_arg)

vehicle 1 created


## Execute Plan

In [18]:
mission_incomplete = True
while mission_incomplete:
    mission_incomplete = uav.act_plan()

vehicle 1: action land is done


KeyboardInterrupt: 

## Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")